In [5]:
import utils
import sympy
import numpy
import itertools

In [8]:
def num_denom (A, symbolic_expr = False):

    sym_den = 0
    sym_num = 0
    s = sympy.Symbol('s')

    if type(A) == utils.mimotf:
        denom   = 1
        num     = 1
        
        for j in range(A.matrix.shape[1]):
            denom = list(numpy.poly1d(denom) * numpy.poly1d(A.matrix[0,j].denominator.coeffs))
            num   = list(numpy.poly1d(num)   * numpy.poly1d(A.matrix[0,j].numerator.coeffs))
            if symbolic_expr == True:
                for n in range(len(denom)):
                    sym_den = (sym_den + denom[len(denom) - n- 1] * s**n).simplify()
                for n in range(len(num)):
                    sym_num = (sym_num + num[len(num) - n- 1] * s**n).simplify()
                return sym_num, sym_den
            else:
                return num, denom
            
    elif type(A) == utils.tf:
        denom = []
        num = []
            
        denom = [list(A.denominator.coeffs)[n] for n in range(len(list(A.denominator.coeffs)))]
        num   = [list(A.numerator.coeffs)[n] for n in range(len(list(A.numerator.coeffs)))]
        if symbolic_expr == True:
            for n in range(len(denom)):
                sym_den = (sym_den + denom[len(denom) - n - 1] * s**n).simplify()
            for n in range(len(num)):
                sym_num = (sym_num + num[len(num) - n - 1] * s**n).simplify()
            return sym_num, sym_den
        else:
            return num, denom
    else:
        sym_num, sym_den = A.as_numer_denom()
        if not symbolic_expr:
            num_poly   = sympy.Poly(sym_num)
            numer      = [float(k) for k in num_poly.all_coeffs()]
            den_poly   = sympy.Poly(sym_den)
            denom      = [float(k) for k in den_poly.all_coeffs()]
            return numer, denom
        else:
            return sym_num, sym_den

def minors(G, order):
    '''
    Returns the order minors of a MIMO tf G.
    '''
    minor = []
    Nrows, Ncols = G.shape
    for rowstokeep in itertools.combinations(range(Nrows), order):
        for colstokeep in itertools.combinations(range(Ncols), order):
            minor.append(utils.det(G[rowstokeep,colstokeep]))
    return minor


def lcm_of_all_minors(G):
    '''
    Returns the lowest common multiple of all minors of G 
    '''
    Nrows, Ncols = G.shape
    lcm = 1
    for i in range(1, min(Nrows, Ncols) + 1, 1):
        allminors = minors(G, i)
        for m in allminors:
            numer, denom = num_denom(m,symbolic_expr = True)
            lcm = sympy.lcm(lcm, denom)
    return lcm


def poles(G):
    '''
    Return the poles of a multivariable transfer function system. Applies
    Theorem 4.4 (p135).

    Parameters
    ----------
    G : sympy or mimotf matrix (n x n)
        The transfer function G(s) of the system.

    Returns
    -------
    pole : array
        List of poles.

    Example
    -------
    >>> s = tf([1,0],[1])
    >>> G = mimotf([[(s - 1) / (s + 2), 4 / (s + 2)],
    ...             [4.5 / (s + 2), 2 * (s - 1) / (s + 2)]])
    >>> poles(G)
    array([-2.])

    '''

    lcm = lcm_of_all_minors(G)
    lcm_poly = sympy.Poly(lcm)
    lcm_coeff = [float(k) for k in lcm_poly.all_coeffs()]
    pole = numpy.roots(lcm_coeff)

    return pole

In [3]:
s = utils.tf([1,0],[1])
G = utils.mimotf([[(s - 1) / (s + 2), 4 / (s + 2)],
            [4.5 / (s + 2), 2 * (s - 1) / (s + 2)], 
            [(s - 2) / (s + 2), 4*s / (s + 2)]])

In [ ]:
type(G) == utils.mimotf

In [9]:
poles(G)

array([-2.])

In [ ]:
s = sp.Symbol('s')
G = 1  / (s + 2) * sp.Matrix([[s - 1,  4],[4.5, 2 * (s - 1)]])

In [ ]:
utils.poles(G)

In [ ]:
G = 1  / (s + 2) * np.matrix([[s - 1,  4],[4.5, 2 * (s - 1)]])